In [1]:
from pathlib import Path, PurePath
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from analysis import read_behaviors, read_moea_fitness, unpickle_moea, disparity_mse
from utility.util_functions import calc_str_disparity, euclidean_distance
from utility.visualize import plot_path, plot_moea_fitness, plot_surrogate_model, plot_str_disparity, plot_moea_fitness_2d
import pickle
import numpy as np
from deap import base, creator
from evolution.simulation import Disparity
import scipy as sp
from scipy import stats
import math
init_notebook_mode(connected=True)


ModuleNotFoundError: DBus works only on linux!


Using TensorFlow backend.


ModuleNotFoundError: DBus works only on linux!


In [2]:
# get the path for all behavioral features
files = sorted(Path('data/results/').glob('transferability_*/behavioral_features.dat'))
fitness_f = sorted(Path('data/results/').glob('transferability_*/fitness.dat'))

In [3]:
columns = [
        'avg_left', 'avg_right',
        's1', 's2', 's3', 's4', 's5', 's6', 's7',
        'area0_percentage',
        'area1_percentage',
        'area2_percentage',
    ]

In [4]:
fitness_f[6]

PosixPath('data/results/transferability_simulation_15/fitness.dat')

In [ ]:
behaviors = read_behaviors([files[6]])[0]

In [ ]:
behaviors.loc[behaviors['genome_id'] == 375].head()

In [ ]:
behaviors.head()

In [ ]:
b_grouped = behaviors.groupby(['genome_id']).filter(lambda x: len(x) > 1).groupby(['genome_id', 'simulation']).first()

In [ ]:
b_grouped.head()

In [ ]:
euclidean_distance(b_grouped[columns].iloc[1], b_grouped[columns].iloc[0])

In [ ]:
for i in range(1, len(b_grouped), 2):
    print('genome_id:', 
          b_grouped[columns].iloc[i].name[0],
          'actual diversity',
          euclidean_distance(
              b_grouped[columns].iloc[i],
              b_grouped[columns].iloc[i-1]
          )
    )

In [5]:
fitness_data = read_moea_fitness([fitness_f[6]])[0]

In [ ]:
fitness_data.sort_values(by=['fitness'])

In [ ]:
fitness_data.head()

In [6]:
hof = unpickle_moea('data/results/transferability_simulation_15/hof')

In [7]:
len(hof)

12

In [8]:
plot_moea_fitness_2d(fitness_data, hof)

PlotlyRequestError: No message

In [ ]:
hof_ids = []
for ind in hof:
    hof_ids.append(ind.key)
    print(ind.key, ind.fitness.values)

In [ ]:
genomes_file = sorted(Path('data/results/transferability_simulation_15/deap_inds').glob('*_genome.pkl'))
transfered_file = sorted(Path('data/results/transferability_simulation_15/deap_inds').glob('*_transformed_genome_.pkl'))
genomes = [unpickle_moea(genome) for genome in genomes_file]
transferred = [unpickle_moea(genome) for genome in transfered_file]

In [ ]:
transfered_file

In [ ]:
len(transfered_file)

In [ ]:
g = sorted(genomes, key=lambda x: x.key)

In [ ]:
t = sorted(transferred, key=lambda x: x.key)

In [ ]:
sim_genome = list(filter(lambda x: x.key == 375, g))[0]

In [ ]:
print(sim_genome.key, sim_genome.fitness.values, sim_genome.sim_time, sim_genome.diversity)

In [ ]:
trans_genome = list(filter(lambda x: x.key == 375, t))[0]

In [ ]:
print(trans_genome.key, trans_genome.fitness.values)

In [ ]:
sim_genome.sim_time

In [ ]:
sim_genomes = sorted(list(filter(lambda x: x.key in hof_ids[:-10], g)), key=lambda x: x.fitness.values[0])
trans_genomes = sorted(list(filter(lambda x: x.key in hof_ids[:-10] t)), key=lambda x: x.fitness.values[0])

In [ ]:
plot_path([sim_genome, trans_genome], 'The discrepancy of traveled trajectories in simulation & reality.')
# plot_path(sim_genomes)

In [ ]:
sim_genome.position

In [ ]:
trans_genome.position

In [ ]:
plot_path(trans_genomes)

In [ ]:
calc_str_disparity(trans_genome.position, sim_genome.position)

In [ ]:
euclidean_distance(trans_genome.features, sim_genome.features)

In [ ]:
euclidean_distance(trans_genome.position, sim_genome.position)

In [ ]:
plot_surrogate_model(fitness_data)

In [ ]:
# merge  
union_t = [(x, y) for x in g for y in t if x.key == y.key]
for s, t in union_t:
    np.array_equal(s, t)
    assert s.key == t.key

In [ ]:
# merge HOF
union_hof = [(x, y) for x in hof for y in t if x.key == y.key]
for s, t in union_hof:
    np.array_equal(s, t)
    assert s.key == t.key

In [ ]:
len(union_hof)

In [ ]:
for s, t in union_hof:
    print(s.key, end=" ")

In [ ]:
str_disparities = [(s.gen,
                    s.key,
                    s.fitness.values[0],
                    s.fitness.values[1],
                    calc_str_disparity(t.position, s.position)
                   ) for s, t in union_t
                  ]

In [ ]:
str_euclidean = [(s.gen,
                    s.key,
                    s.fitness.values[0],
                    s.fitness.values[1],
                    calc_str_disparity(t.position, s.position)
                   ) for s, t in union_t
                ]

In [ ]:
str_disparities_hof = [(s.gen,
                    s.key,
                    s.fitness.values[0],
                    s.fitness.values[1],
                    calc_str_disparity(t.position, s.position)
                   ) for s, t in union_hof
                  ]

In [ ]:
sorted(str_disparities, key=lambda x: x[0])

In [ ]:
np.array([str_disparity[1] for str_disparity in str_disparities])
np.array([real_disparity[2] for real_disparity in str_disparities])

filter_disparities = list(filter(lambda x: x[2] < 5, str_disparities))
filter_disparities[:4]

In [ ]:
plot_str_disparity(str_disparities)

In [ ]:
actual = np.array([a[4] for a in str_disparities_hof])
approximated = np.array([a[3] for a in str_disparities_hof])

In [ ]:
approximated

In [ ]:
actual

In [ ]:
disparity_mse(approximated, actual)

In [ ]:
actual = []
approximated = []
for s, t in union_t:
    approximated.append(s.fitness.values[1])
    actual.append(calc_str_disparity(t.position, s.position))

In [ ]:
stats.pearsonr(actual, approximated)

In [ ]:
str_euclidean

In [ ]:
distance = 0
for i in range(0, len(trans_genome.position)-1):
    distance += math.sqrt(((sim_genome.position[i][0] - 
                            sim_genome.position[i][1])**2) + 
                          ((sim_genome.position[i+1][1] - 
                            sim_genome.position[i+1][1])**2))

In [ ]:
def traveled_distance(genome):
    distance = 0
    for i in range(0, len(genome.position)-1):
        distance += math.sqrt(((genome.position[i][0] - 
                                genome.position[i][1])**2) + 
                              ((genome.position[i+1][1] - 
                                genome.position[i+1][1])**2))
    return distance

In [ ]:
traveled_distance(trans_genome)

In [ ]:
traveled_distance(sim_genome)